In [1]:
from genericpath import isfile
import rasterio
import os, glob
import numpy as np
import geopandas as gpd
import rasterio.features
from tqdm.notebook import tqdm


def create_mask(img_path, shp_path, value_mask=None):
    with rasterio.open(img_path) as src:
        height, width = src.height, src.width
        tr = src.transform
        crs_img = src.crs
        
    df = gpd.read_file(shp_path)
    # check epsg
    if df.crs.to_string() != crs_img.to_string():
        df = df.to_crs(epsg=str(crs_img.to_epsg()))
    shapes = df['geometry']
    mask = rasterio.features.rasterize(shapes, out_shape=(height, width), transform=tr)
    return np.array([mask])



list_fp_rm_by_shape = [
    "20220826_065957_ssc3_u0001_visual.tif",
    "20220821_070453_ssc12_u0001_visual.tif",
    "20220820_103902_ssc6_u0001_visual.tif",
    "20220813_070157_ssc12_u0002_visual.tif",
    "20220821_070453_ssc12_u0002_visual.tif",
    "20220819_102529_ssc8_u0002_visual.tif",
    "20220818_073620_ssc1_u0002_visual.tif",
    "20220814_103841_ssc10_u0002_visual.tif",
    "20220814_103841_ssc10_u0001_visual.tif",
    "20220807_064607_ssc4_u0002_visual.tif",
    "20220812_070039_ssc4_u0002_visual.tif",
    "20220813_070157_ssc12_u0001_visual.tif"
]
dir_mask = "/home/skm/SKM16/Work/OpenLand/all_tif/Water_of_Openland_1666456777_loadmodel"
dir_shape_remove = "/home/skm/public_mount/DucAnhtmp/HIEUUUUUUUUUUUUUUU/xoa"
dir_shape_add = "/home/skm/public_mount/DucAnhtmp/HIEUUUUUUUUUUUUUUU/them"

out_dir_mask = "/home/skm/SKM16/Work/OpenLand/Result_Final/Water/tmp"

list_fp_shp_xoa = [os.path.basename(fp) for fp in glob.glob(os.path.join(dir_shape_remove, "*.shp"))]
list_fp_shp_add = [os.path.basename(fp) for fp in glob.glob(os.path.join(dir_shape_add, "*.shp"))]

list_fname_shp_fix = np.unique(list_fp_shp_xoa + list_fp_shp_add)
# print(list_fname_shp_fix)

for fname_shp in tqdm(list_fname_shp_fix):
    fname_tif = fname_shp.replace('.shp','.tif')
    out_fp = os.path.join(out_dir_mask, fname_tif)
    
    fp_mask = os.path.join(dir_mask, fname_tif)
    fp_shp_xoa = None
    fp_shp_them = None
    
    with rasterio.open(fp_mask) as src:
        meta = src.meta
        mask = src.read()
    
    # cap nhat lai shape file xoa
    if os.path.isfile(os.path.join(dir_shape_remove, fname_shp)):
        fp_shp_xoa = os.path.join(dir_shape_remove, fname_shp)
    # cap nhat lai shape file add
    if os.path.isfile(os.path.join(dir_shape_add, fname_shp)):
        fp_shp_them = os.path.join(dir_shape_add, fname_shp)
    
    if fp_shp_xoa:
        mask_draw_xoa = create_mask(fp_mask, fp_shp_xoa, value_mask=None)
        mask_draw_xoa = 1 - mask_draw_xoa
    else:
        mask_draw_xoa = np.ones_like(mask)
    
    if fp_shp_them:
        mask_draw_them = create_mask(fp_mask, fp_shp_them, value_mask=None)
    else:
        mask_draw_them = np.zeros_like(mask)
        

    mask_final = mask*mask_draw_xoa + mask_draw_them
    mask_final[mask_final != 0] = 1
    with rasterio.open(out_fp, 'w', **meta) as dst:
        dst.write(mask_final)

  0%|          | 0/12 [00:00<?, ?it/s]